In [3]:
# import tensorflow as tf
import numpy as np
import h2o
import os
import pandas as pd
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder

import category_encoders as ce


# from .feature_engineering_filter import Find_correct_port

/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [36]:
# Check if an H2O cluster is already running
if h2o.cluster().is_running():
    print("An H2O cluster is already running. Shutting it down...")
    h2o.shutdown(prompt=True)

h2o.init(max_mem_size="4g")

/var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/ipykernel_94604/3287064515.py:4: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=True)


An H2O cluster is already running. Shutting it down...
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.1" 2021-10-19; OpenJDK Runtime Environment Temurin-17.0.1+12 (build 17.0.1+12); OpenJDK 64-Bit Server VM Temurin-17.0.1+12 (build 17.0.1+12, mixed mode, sharing)
  Starting server from /Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.venv/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/tmpoqt649v2
  JVM stdout: /var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/tmpoqt649v2/h2o_kristofferseyffarth_started_from_python.out
  JVM stderr: /var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/tmpoqt649v2/h2o_kristofferseyffarth_started_from_python.err
  Server is running at http://127.0.0.1:54341
Connecting to H2O server at http://127.0.0.1:54341 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 21 days
H2O_cluster_name:,H2O_from_python_kristofferseyffarth_bc70ad
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [38]:
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")

In [39]:
train_data["time"] = pd.to_datetime(train_data["time"])
test_data["time"] = pd.to_datetime(test_data["time"])

In [63]:
train_data_preprocessed = train_data.copy()
train_data_preprocessed[train_data_preprocessed["cog"] >= 360] = np.nan
train_data_preprocessed[train_data_preprocessed["sog"] >= 1.023] = np.nan
train_data_preprocessed[train_data_preprocessed["rot"] == -128] = np.nan
train_data_preprocessed[train_data_preprocessed["heading"] == 511] = np.nan

train_data_preprocessed["cog"] = (
    train_data_preprocessed["cog"].groupby(train_data_preprocessed["vesselId"]).ffill()
)
train_data_preprocessed["sog"] = (
    train_data_preprocessed["sog"].groupby(train_data_preprocessed["vesselId"]).ffill()
)
train_data_preprocessed["rot"] = (
    train_data_preprocessed["rot"].groupby(train_data_preprocessed["vesselId"]).ffill()
)
train_data_preprocessed["heading"] = (
    train_data_preprocessed["heading"]
    .groupby(train_data_preprocessed["vesselId"])
    .ffill()
)
print(train_data_preprocessed["etaRaw"].head())

pattern = r"^\d{2}-\d{2} \d{2}:\d{2}$"
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].where(
    train_data_preprocessed["etaRaw"].str.match(pattern, na=False), np.nan
)
print(train_data_preprocessed["etaRaw"].head())


train_data_preprocessed["etaRaw"] = train_data_preprocessed.groupby("vesselId")[
    "etaRaw"
].ffill()

print(train_data_preprocessed["etaRaw"].head())


train_data_preprocessed["navstat"] = train_data_preprocessed["navstat"].astype(
    "category"
)

train_data_preprocessed = train_data_preprocessed.dropna(subset=["etaRaw"])

print(train_data_preprocessed["etaRaw"].head())

# # Replace '00-' in etaRaw with the corresponding month and day from the 'time' column
# train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
#     train_data_preprocessed["etaRaw"].str.contains("00-", na=False),
#     train_data_preprocessed["time"].dt.strftime(
#     "%m-%d"
# )
#     + " "
#     + train_data_preprocessed["etaRaw"].str[6:],
# )

# Replace '00-' in etaRaw with the corresponding month and day from the 'time' column
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("00-", na=False),
    "01"
    + train_data_preprocessed["etaRaw"].str[2:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("-00", na=False),
    train_data_preprocessed["etaRaw"].str[:2]
    +"-01"
    + train_data_preprocessed["etaRaw"].str[5:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains(":60", na=False),
    train_data_preprocessed["etaRaw"].str[:9]+"59",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("60:", na=False),
    train_data_preprocessed["etaRaw"].str[:6]+"01:00",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("24:", na=False),
    train_data_preprocessed["etaRaw"].str[:6]+"23:59"
)


train_data_preprocessed["etaRaw"] = pd.to_datetime(
    train_data_preprocessed["time"].dt.year.astype(str)
    + "-"
    + train_data_preprocessed["etaRaw"]
    + ":00",
    format="%Y-%m-%d %H:%M:%S",
)


print(train_data_preprocessed["etaRaw"].head())
train_data_preprocessed["seconds_to_eta"] = (
    train_data_preprocessed["time"] - train_data_preprocessed["etaRaw"]
).dt.total_seconds()

train_data_preprocessed = train_data_preprocessed.drop(columns=["etaRaw"])

0    01-09 23:00
1    12-29 20:00
2            NaN
3    12-31 20:00
4            NaN
Name: etaRaw, dtype: object
0    01-09 23:00
1    12-29 20:00
2            NaN
3    12-31 20:00
4            NaN
Name: etaRaw, dtype: object
0    01-09 23:00
1    12-29 20:00
2            NaN
3    12-31 20:00
4            NaN
Name: etaRaw, dtype: object
0    01-09 23:00
1    12-29 20:00
3    12-31 20:00
5    12-20 02:40
7    12-31 18:30
Name: etaRaw, dtype: object
0   2024-01-09 23:00:00
1   2024-12-29 20:00:00
3   2024-12-31 20:00:00
5   2024-12-20 02:40:00
7   2024-12-31 18:30:00
Name: etaRaw, dtype: datetime64[ns]


In [ ]:
train_latitude_radians = np.deg2rad(train_data["latitude"])
train_longitude_radians = np.deg2rad(train_data["longitude"])
train_cog_radians = np.deg2rad(train_data["longitude"])
train_heading_radians = np.deg2rad(train_data["longitude"])
train_hour = np.deg2rad(train_data["time"].dt.hour * 360 / 24)
train_day = np.deg2rad(train_data["time"].dt.day * 360 / 30)
train_month = np.deg2rad(train_data["time"].dt.month * 360 / 12)

train_latitude_sin = np.sin(train_latitude_radians)
train_latitude_cos = np.cos(train_latitude_radians)

train_longitude_sin = np.sin(train_longitude_radians)
train_longitude_cos = np.cos(train_longitude_radians)

train_cog_sin = np.sin(train_cog_radians)
train_cog_cos = np.cos(train_cog_radians)

train_heading_sin = np.sin(train_heading_radians)
train_heading_cos = np.cos(train_heading_radians)

train_hour_sin = np.sin(train_hour)
train_hour_cos = np.cos(train_hour)

train_day_sin = np.sin(train_day)
train_day_cos = np.cos(train_day)

train_month_sin = np.sin(train_month)
train_month_cos = np.cos(train_month)


train_data_preprocessed["latitude_sin"] = train_latitude_sin
train_data_preprocessed["latitude_cos"] = train_latitude_cos
train_data_preprocessed["longitude_sin"] = train_longitude_sin
train_data_preprocessed["longitude_cos"] = train_longitude_cos
train_data_preprocessed["cog_sin"] = train_cog_sin
train_data_preprocessed["cog_cos"] = train_cog_cos
train_data_preprocessed["heading_sin"] = train_heading_sin
train_data_preprocessed["heading_cos"] = train_heading_cos

train_data_preprocessed["hour_sin"] = train_hour_sin
train_data_preprocessed["hour_cos"] = train_hour_cos
train_data_preprocessed["day_sin"] = train_day_sin
train_data_preprocessed["day_cos"] = train_day_cos
train_data_preprocessed["month_sin"] = train_month_sin
train_data_preprocessed["month_cos"] = train_month_cos


train_data_preprocessed = train_data_preprocessed.drop(
    columns=["latitude", "longitude", "cog", "heading", "portId"], axis=1
)
print(train_data_preprocessed.head())

                 time   sog  rot navstat       etaRaw  \
0 2024-01-01 00:00:25   0.7    0       0  01-09 23:00   
1 2024-01-01 00:00:36   0.0   -6       1  12-29 20:00   
2 2024-01-01 00:01:45  11.0    0       0  01-02 09:00   
3 2024-01-01 00:03:11   0.0    0       1  12-31 20:00   
4 2024-01-01 00:03:51  19.7    0       0  01-25 12:00   

                   vesselId  latitude_sin  latitude_cos  longitude_sin  \
0  61e9f3a8b937134a3c4bfdf7     -0.569906      0.821710      -0.846670   
1  61e9f3d4b937134a3c4bff1f      0.154614      0.987975      -0.983189   
2  61e9f436b937134a3c4c0131      0.631903      0.775048      -0.972271   
3  61e9f3b4b937134a3c4bfe77     -0.565138      0.824996       0.484494   
4  61e9f41bb937134a3c4c0087      0.586143      0.810208      -0.103077   

   longitude_cos   cog_sin   cog_cos  heading_sin  heading_cos  
0       0.532118 -0.846670  0.532118    -0.846670     0.532118  
1       0.182589 -0.983189  0.182589    -0.983189     0.182589  
2       0.233858 

In [40]:
def Last_known_location_training_data(data: pd.DataFrame) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data
    """

    shift_length = 10

    grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))

    grouped_data["time_diff"] = (
        grouped_data["time"].diff(-shift_length).dt.total_seconds().abs().fillna(0)
    )

    original_time_and_id = grouped_data[
        [
            "time",
            "vesselId",
            "latitude_sin",
            "latitude_cos",
            "longitude_sin",
            "longitude_cos",
        ]
    ]

    shifted_data = grouped_data.shift(shift_length)
    shifted_data[
        [
            "last_latitude_sin",
            "last_latitude_cos",
            "last_longitude_sin",
            "last_longitude_cos",
        ]
    ] = shifted_data[["latitude_sin", "latitude_cos", "longitude_sin", "longitude_cos"]]

    shifted_data[
        [
            "time",
            "vesselId",
            "latitude_sin",
            "latitude_cos",
            "longitude_sin",
            "longitude_cos",
        ]
    ] = original_time_and_id[
        [
            "time",
            "vesselId",
            "latitude_sin",
            "latitude_cos",
            "longitude_sin",
            "longitude_cos",
        ]
    ]

    # Drops all values with nan values
    result = shifted_data.dropna().reset_index(drop=True)

    return result

In [41]:
train_data_shifted_df = Last_known_location_training_data(train_data_preprocessed)

train_data_shifted_df = train_data_shifted_df.drop(["time", "etaRaw"], axis=1)

train_data_shifted = h2o.H2OFrame(train_data_shifted_df)

/var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/ipykernel_94604/824791856.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [42]:
train_data_shifted_without_validation, validation_data_shifted = (
    train_data_shifted.split_frame(ratios=[0.8], seed=42)
)

In [43]:
def append_last_known_data_test(
    test_data: pd.DataFrame, known_data: pd.DataFrame
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data
    """

    if not test_data["vesselId"].isin(known_data["vesselId"]).all():
        missing_vessels = test_data[
            ~test_data["vesselId"].isin(known_data["vesselId"])
        ]["vesselId"].unique()
        raise ValueError(
            f"The following vesselIds are missing in known_data: {missing_vessels}"
        )
    print(
        test_data[~test_data["vesselId"].isin(known_data["vesselId"])][
            "vesselId"
        ].unique()
    )

    grouped_data = (
        known_data.sort_values("time")
        .groupby("vesselId")
        .tail(1)
        .reset_index(drop=True)
    )
    original_time = test_data[["time"]]
    test_data = test_data.drop("time", axis=1)

    result = pd.merge(test_data, grouped_data, how="left", on="vesselId")

    result["time_diff"] = (original_time["time"] - result["time"]).dt.total_seconds()

    return result

In [44]:
test_data_with_last_known_df = append_last_known_data_test(
    test_data, train_data_preprocessed
)
test_data_with_last_known_df[
    [
        "last_latitude_sin",
        "last_latitude_cos",
        "last_longitude_sin",
        "last_longitude_cos",
    ]
] = test_data_with_last_known_df[
    [
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ]
]
test_data_with_last_known_df = test_data_with_last_known_df.drop(
    columns=[
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ],
    axis=1,
)

test_data_with_last_known = h2o.H2OFrame(test_data_with_last_known_df)
print(test_data_with_last_known_df.head())

[]
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
   ID                  vesselId  scaling_factor                time   sog  \
0   0  61e9f3aeb937134a3c4bfe3d             0.3 2024-05-07 23:48:16   0.0   
1   1  61e9f473b937134a3c4c02df             0.3 2024-05-07 23:57:16   0.0   
2   2  61e9f469b937134a3c4c029b             0.3 2024-05-07 23:59:08  18.7   
3   3  61e9f45bb937134a3c4c0221             0.3 2024-05-07 23:52:34   0.1   
4   4  61e9f38eb937134a3c4bfd8d             0.3 2024-05-07 23:51:29   0.3   

   rot navstat       etaRaw   cog_sin   cog_cos  heading_sin  heading_cos  \
0    0       5  05-06 10:45 -0.989010  0.147846    -0.989010     0.147846   
1    0       5  05-01 23:00  0.863429 -0.504471     0.863429    -0.504471   
2    0       0  05-08 12:45  0.187086  0.982343     0.187086     0.982343   
3    0       1  05-07 01:15  0.124723 -0.992192     0.124723    -0.992192   
4    0       2  05-09 04:00 -0.106612  0.994301    -0.

In [45]:
test_data_with_last_known_df.to_csv("../Datasets/test_data_with_last_known.csv")

In [46]:
features_lat = [
    "vesselId",
    "cog_sin",
    "cog_cos",
    "sog",
    "rot",
    "heading_sin",
    "heading_cos",
    "navstat",
    "time_diff",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
]
features_long = [
    "vesselId",
    "cog_sin",
    "cog_cos",
    "sog",
    "rot",
    "heading_sin",
    "heading_cos",
    "navstat",
    "time_diff",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    "latitude_sin",
    "latitude_cos",
]
target_long_sin = "longitude_sin"
target_long_cos = "longitude_cos"
target_lat_sin = "latitude_sin"
target_lat_cos = "latitude_cos"

In [47]:
params = {
    "ntrees": 300,  # Maximum number of trees
    "max_depth": 6,  # Reduced maximum depth of each tree
    "min_rows": 20,  # Increased minimum number of rows per leaf
    "learn_rate": 0.05,  # Learning rate
    "sample_rate": 0.8,  # Reduced row sample rate per tree
    "col_sample_rate": 0.8,  # Reduced column sample rate per tree
    "reg_lambda": 1.0,  # L2 regularization term
    "reg_alpha": 0.1,  # L1 regularization term
    "seed": 42,  # Random seed for reproducibility
    "stopping_rounds": 10,  # Early stopping rounds
    "stopping_metric": "AUTO",  # Metric for early stopping
    "stopping_tolerance": 0.001,  # Tolerance for early stopping
}


gbm_lat_sin = h2o.estimators.H2OXGBoostEstimator(
    **params
    # ntrees=200,  # Maximum number of trees
    # max_depth=10,  # Maximum depth of each tree
    # min_rows=10,  # Minimum number of rows per leaf
    # learn_rate=0.05,  # Learning rate
    # sample_rate=0.9,  # Row sample rate per tree
    # col_sample_rate=0.9,  # Column sample rate per tree
    # reg_lambda=1.0,  # L2 regularization term
    # reg_alpha=0.1,  # L1 regularization term
    # seed=42,  # Random seed for reproducibility
)
gbm_lat_cos = h2o.estimators.H2OXGBoostEstimator(
    **params
    # ntrees=200,  # Maximum number of trees
    # max_depth=10,  # Maximum depth of each tree
    # min_rows=10,  # Minimum number of rows per leaf
    # learn_rate=0.05,  # Learning rate
    # sample_rate=0.9,  # Row sample rate per tree
    # col_sample_rate=0.9,  # Column sample rate per tree
    # reg_lambda=1.0,  # L2 regularization term
    # reg_alpha=0.1,  # L1 regularization term
    # seed=42,  # Random seed for reproducibility
)
gbm_long_sin = h2o.estimators.H2OXGBoostEstimator(
    **params
    # ntrees=200,  # Maximum number of trees
    # max_depth=10,  # Maximum depth of each tree
    # min_rows=10,  # Minimum number of rows per leaf
    # learn_rate=0.05,  # Learning rate
    # sample_rate=0.9,  # Row sample rate per tree
    # col_sample_rate=0.9,  # Column sample rate per tree
    # reg_lambda=1.0,  # L2 regularization term
    # reg_alpha=0.1,  # L1 regularization term
    # seed=42,  # Random seed for reproducibility
)
gbm_long_cos = h2o.estimators.H2OXGBoostEstimator(
    **params
    # ntrees=200,  # Maximum number of trees
    # max_depth=10,  # Maximum depth of each tree
    # min_rows=10,  # Minimum number of rows per leaf
    # learn_rate=0.05,  # Learning rate
    # sample_rate=0.9,  # Row sample rate per tree
    # col_sample_rate=0.9,  # Column sample rate per tree
    # reg_lambda=1.0,  # L2 regularization term
    # reg_alpha=0.1,  # L1 regularization term
    # seed=42,  # Random seed for reproducibility
)

# gbm_cog = h2o.estimators.H2OXGBoostEstimator()
# gbm_sog = h2o.estimators.H2OXGBoostEstimator()
# gbm_rot = h2o.estimators.H2OXGBoostEstimator()
# gbm_heading = h2o.estimators.H2OXGBoostEstimator()
# gbm_navstat = h2o.estimators.H2OXGBoostEstimator()
# # gbm_etaRaw = h2o.esti#mators.H2OXGBoostEstimator() #Remove etaRaw because it requires preprocessing
# # gbm_portId = h2o.estimators.H2OXGBoostEstimator()

In [48]:
print(train_data_shifted_without_validation.head())

  sog    rot    navstat  vesselId                    latitude_sin    latitude_cos    longitude_sin    longitude_cos    cog_sin    cog_cos    heading_sin    heading_cos        time_diff    last_latitude_sin    last_latitude_cos    last_longitude_sin    last_longitude_cos
 17.1     -6          0  61e9f38eb937134a3c4bfd8b        0.140983        0.990012         0.973701         0.227828   0.97661    0.215018       0.97661        0.215018  12529                       0.130589             0.991437              0.97661               0.215018
 17.3      5          0  61e9f38eb937134a3c4bfd8b        0.141929        0.989877         0.973407         0.229084   0.976277   0.216524       0.976277       0.216524  12396                       0.13179              0.991278              0.976277              0.216524
 16.9      5          0  61e9f38eb937134a3c4bfd8b        0.142588        0.989782         0.973112         0.230331   0.975894   0.218245       0.975894       0.218245  11929             

In [49]:
gbm_lat_sin.train(
    x=features_lat,
    y=target_lat_sin,
    training_frame=train_data_shifted_without_validation,
)

xgboost Model Build progress: |

/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.venv/lib/python3.12/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: early stopping is enabled but neither score_tree_interval or score_each_iteration are defined. Early stopping will not be reproducible!
  warnings.warn(mesg["message"], RuntimeWarning)


███████████████████████████████████████████████ (cancelled)  94%


H2OJobCancelled: Job<$03017f00000146d4ffffffff$_b2cf6635c85c5db671051abc015d4734> was cancelled by the user.

In [ ]:
gbm_lat_cos.train(
    x=features_lat,
    y=target_lat_cos,
    training_frame=train_data_shifted_without_validation,
)

In [ ]:
performance_lat_sin = gbm_lat_sin.model_performance(test_data=validation_data_shifted)
performance_lat_cos = gbm_lat_cos.model_performance(test_data=validation_data_shifted)


# Print the performance metrics
print(performance_lat_sin)
print(performance_lat_cos)

In [ ]:
gbm_long_sin.train(
    x=features_long,
    y=target_long_sin,
    training_frame=train_data_shifted_without_validation,
)

In [ ]:
gbm_long_cos.train(
    x=features_long,
    y=target_long_cos,
    training_frame=train_data_shifted_without_validation,
)

In [ ]:
performance_long_sin = gbm_long_sin.model_performance(test_data=validation_data_shifted)
performance_long_cos = gbm_long_cos.model_performance(test_data=validation_data_shifted)

print(performance_long_sin)
print(performance_long_cos)

In [ ]:
lat_predictions_sin = gbm_lat_sin.predict(test_data_with_last_known)
lat_predictions_cos = gbm_lat_cos.predict(test_data_with_last_known)

test_data_with_predicted_lat = test_data_with_last_known
test_data_with_predicted_lat["latitude_sin"] = lat_predictions_sin
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

long_predictions_sin = gbm_long_sin.predict(test_data_with_predicted_lat)
long_predictions_cos = gbm_long_cos.predict(test_data_with_predicted_lat)

In [ ]:
# Convert sine and cosine values back to radians
lat_predictions_sin = lat_predictions_sin.as_data_frame()
lat_predictions_cos = lat_predictions_cos.as_data_frame()
long_predictions_sin = long_predictions_sin.as_data_frame()
long_predictions_cos = long_predictions_cos.as_data_frame()


lat_predictions_radians = np.arctan2(lat_predictions_sin, lat_predictions_cos)
long_predictions_radians = np.arctan2(long_predictions_sin, long_predictions_cos)

# Convert radians to degrees
lat_predictions_degrees = np.rad2deg(lat_predictions_radians)
long_predictions_degrees = np.rad2deg(long_predictions_radians)

# Print the first few rows to verify the conversion
print(lat_predictions_degrees.head())
print(long_predictions_degrees.head())

In [ ]:
def create_prediction_visualization_data(validation_data):
    lat_val_sin = gbm_lat_sin.predict(validation_data)
    lat_val_cos = gbm_lat_cos.predict(validation_data)
    long_val_sin = gbm_long_sin.predict(validation_data)
    long_val_cos = gbm_long_cos.predict(validation_data)

    lat_val_sin = lat_val_sin.as_data_frame()
    lat_val_cos = lat_val_cos.as_data_frame()
    long_val_sin = long_val_sin.as_data_frame()
    long_val_cos = long_val_cos.as_data_frame()

    validation_data = validation_data.as_data_frame()

    lat_val_radians = np.arctan2(lat_val_sin, lat_val_cos)
    long_val_radians = np.arctan2(long_val_sin, long_val_cos)

    evaluation_lat_radians = np.arctan2(
        validation_data["latitude_sin"], validation_data["latitude_cos"]
    )
    evaluation_long_radians = np.arctan2(
        validation_data["longitude_sin"], validation_data["longitude_cos"]
    )

    # Convert radians to degrees
    lat_val_degrees = np.rad2deg(lat_val_radians)
    long_val_degrees = np.rad2deg(long_val_radians)

    evaluation_lat_degrees = np.rad2deg(evaluation_lat_radians)
    evaluation_long_degrees = np.rad2deg(evaluation_long_radians)

    eval_predictions = pd.concat([lat_val_degrees, long_val_degrees], axis=1)

    eval_actual = pd.concat([evaluation_lat_degrees, evaluation_long_degrees], axis=1)

    eval_predictions.columns = ["latitude_predicted", "longitude_predicted"]
    eval_actual.columns = ["latitude", "longitude"]

    eval = pd.DataFrame()
    eval[["latitude_predicted", "longitude_predicted"]] = eval_predictions
    eval[["latitude", "longitude"]] = eval_actual
    eval[["vesselId", "time"]] = validation_data[["vesselId", "time"]]
    eval.to_csv("eval_predictions.csv")


create_prediction_visualization_data(validation_data_shifted)

In [ ]:
predictions = pd.concat([lat_predictions_degrees, long_predictions_degrees], axis=1)
predictions.columns = ["latitude_predicted", "longitude_predicted"]

In [ ]:
predictions["ID"] = test_data["ID"]
predictions = predictions[["ID", "longitude_predicted", "latitude_predicted"]]

In [ ]:
print(predictions.columns)

In [ ]:
predictions.to_csv("predictions.csv", index=False)